In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import transformerblock as tran
import GCN
import os
import matplotlib.pyplot as plot

Sembla que els .h m'estan donant massa problemes, si els csv surten bé i es poden importar correctament seria l'hostia


In [2]:
# dades = pd.read_hdf('datasets/metr-la.h5')
# freq = 0
# dset = dades['773869']


second = pd.read_csv('datasets/PEMS-BAY.csv')
dset2 = second['400001']

In [3]:
matrix = pd.read_pickle('datasets/adj_mx_PEMS-BAY.pkl')
adj_matrx = matrix

"""
def load_graph_data(pkl_filename):
    sensor_ids, sensor_id_to_ind, adj_mx = load_pickle(pkl_filename)
    return sensor_ids, sensor_id_to_ind, adj_mx
"""

adj_mtx = sp.coo_matrix(adj_matrx[2])
# grafo = nx.Graph(adj_mtx)


En els papers diuen que utilitzen un model per fer la espacial que es basa en facer una regularització laplaciana en les funcions de perdues

segons el paper: L = L0 + sigmaLreg

 Lreg =sumatori i,j Aij ||f(Xi) − f(Xj )^2|| = f(X)^T ∆f(X).

L0 es la perdua supervisada

El Xout de la espacial ha de ser
Xout = σ(D˜ ^-1/2 A˜ D˜ ^−1/2 Xin W) 
la sigma aes una ReLU(·)

A~ = A + In on  a es la matriu d'adjecencia i l'altre la identitat

In [4]:
####Normalitzar adjecencia, funcion treta d'un paper

def degree_mat(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)) # aqui seria d'on sortirien els valors de la diagonal 
    # d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    # d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    # d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    # return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    return rowsum


In [5]:
def ref_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)) # aqui seria d'on sortirien els valors de la diagonal 
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

In [6]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

In [7]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)

Per definir el graf del sistema ho farem de la seguent manera

Vertex hidden value -> Valor Q dels sensors Edge value -> VAlor de l'adjecencia de la matriu R

Funcions del message passing: Update -> suma del valor anterior Agregació -> convolució del graf W -> el valor de la funció Xout

x' = Wx'i + W Sumatori(xj / sqrt(di*dj)) per j pertanyent a N(i)U(i)

In [8]:
adj_mtx_csr = adj_mtx.tocsr()
adj_csr_rdx = adj_mtx_csr[00:10,00:10]
D = degree_mat(adj_csr_rdx) # matriu que ens mostra el nivell de conexió del sensor
A = ref_adj(adj_csr_rdx) # matriu d'adjacencies ajustada i refinada
A_tp = sparse_to_tuple(A)
Ds = sp.csr_matrix(D)
D_tp = sparse_to_tuple(Ds)
#com que es la pemsbay es el dataset second
hidden_value_all= second #matriu en forma de fila 0 index nom dels sensors i cap avall en les columnes el valor q de cada sensor cada 5 minuts

#####seria una bona idea treballar amb 10 sensors per ara, k es més visual

hidden_value = hidden_value_all.iloc[:1, 1:11]

hidden_A  = hidden_value.to_numpy()
hidden_valuem = sp.csr_matrix(hidden_value)
hidden_value = preprocess_features(hidden_valuem)

#en la primera instancia del temps els valors q de tots els sensors
# Hs = sp.csr_matrix(hidden_value)
# H_tp = sparse_to_tuple(Hs)
# graph = nx.from_scipy_sparse_matrix(A.tocsc())
# print(nx.edges(graph))

In [9]:
def gen_tensors(mtx):
    idx, idy, val = sp.find(mtx)
    full_indices = tf.stack([idx, idy], axis=1)
    tamany = mtx.shape
    full_indices=tf.cast(full_indices,dtype=tf.int64)
    sparse = tf.sparse.SparseTensor(full_indices,val,tamany)
    sparse=tf.sparse.reorder(sparse)
    # dense = tf.sparse.to_dense(sparse)
    return(sparse)

In [10]:
def message_passing(A,D,hidden):
    A64 = tf.cast(A,dtype=tf.float64)
    A_Hidd = tf.stack([A64,hidden])
    D_layyer = layers.Dense()
    output = D_layyer(A_Hidd)

    #aggregation of messages
    aggr = tf.math.segment_sum(output,D)
    return(aggr)


In [11]:
def one_hop(F, A):
    Fp =[]
    for i in range(A[0].size):
        aux = F[i]*A[i][0]
        Fp.append(np.sum(aux))
    return(Fp)

A_d = A.todense()
passe1 = one_hop(hidden_A[0],A_d)
one_hop(passe1,A_d)

[71.4, 67.8, 125.39743, 67.4, 54.39285, 66.6, 43.50115, 68.0, 66.8, 69.0]

In [12]:
def Mask_A(mat):
    A_mask = mat
    for i in range(mat.shape[0]):
        for j in range(mat[i].size):
            if mat[i,j]> 0:
                A_mask[i,j]=1
    return(A_mask)


In [13]:
def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []
    fnn_layers.append(layers.BatchNormalization())

    for units in hidden_units:
        fnn_layers.append(layers.Dense(units, activation=tf.nn.relu))
        fnn_layers.append(layers.Dropout(dropout_rate))
        
    fnn_layers.append(layers.Dense(units, activation=tf.nn.relu))
    return keras.Sequential(fnn_layers, name=name)

In [14]:

#quan crido la gcn els inputs ahn d'estar en una sola entrada de manera
# node_features les features de cada node [num_nodes, num_features]
# edges matriu adjacencia enmascarada per que siguin 1 i 0 i es marqui adjacencia pura, ha de ser sparse matrix
# la aquesta ha de ser shape [2, num_edges]
# edge wheights nunmpy array amb els pesos del edges [num_edges]
A_mask = Mask_A(A.tocsr().toarray()) #aquesta funcio enmascara la adjecencia normalizada en si o no, el pesos estan guardats en altres matrius
A_mask = sp.csr_matrix(A_mask)
keys_a =[]

for i in range(A_mask.toarray()[0].size):
    for j in range(A_mask[i].size):
        keys_a.append(i)
a_keys = A_mask.indices
keys_a = np.array(keys_a)
edges = np.array([a_keys,keys_a])

hidden_B = tf.cast(hidden_A,dtype=tf.float32)
hidden_C = tf.reshape(hidden_B, [10,1])
a_p = A.tocsr()

ffn1 = create_ffn([1], 0.2, name="preprocess")
node_representations = ffn1(hidden_C)


In [15]:
class convolutions_passing(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        hidden_units,
        aggregation_type="sum",
        combination_type="gru",
        dropout_rate=0.2,
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        edges, edge_weights = graph_info
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GCN.GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create the second GraphConv layer.
        self.conv2 = GCN.GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv2",
        )
        # Create a postprocess layer.
        self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        # self.compute_logits = layers.Dense(units=num_classes, name="logits")

    def call(self, input_node_features):
        # Preprocess the node_features to produce node representation
        x = self.preprocess(input_node_features)
        # Apply the first graph conv layer.
        x1 = self.conv1((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x1 + x
        # Apply the second graph conv layer.
        x2 = self.conv2((x, self.edges, self.edge_weights))
        # Skip connection.
        x = x2 + x
        # Postprocess node embedding.
        x = self.postprocess(x)
        # Fetch node embeddings for the input node_indices.
        #node_embeddings = tf.gather(x, input_node_indices)
        # Compute logits
        return x


In [16]:
#ja tenim el model construit, ara es qüestió de entrenar-lo i mirar de que estigui apunt
hidden_value = hidden_value_all.iloc[:1, 1:]
hidden_D  = hidden_value.to_numpy()
hidden_E = tf.cast(hidden_D,dtype=tf.float32)
hidden_F = tf.reshape(hidden_E, [hidden_D.size,1])
node_rep=ffn1(hidden_F)

adj_mtx_csr = adj_mtx.tocsr()
A = ref_adj(adj_mtx_csr)

A_mask = Mask_A(A.tocsr().toarray()) #aquesta funcio enmascara la adjecencia normalizada en si o no, el pesos estan guardats en altres matrius
A_mask = sp.csr_matrix(A_mask)
keys_a =[]
for i in range(A_mask.toarray()[0].size):
    for j in range(A_mask[i].size):
        keys_a.append(i)
a_keys = A_mask.indices
keys_a = np.array(keys_a)
edges = np.array([a_keys,keys_a])
preprocess = create_ffn([1], 0.2, name="preprocess")
a_p = A.tocsr()
tf.print(edges.shape)

(2, 2694)


In [17]:
hidden_value_next = hidden_value_all.iloc[1:2, 1:]
hidden_D_next  = hidden_value_next.to_numpy()
hidden_E_next = tf.cast(hidden_D_next,dtype=tf.float32)
hidden_F_next = tf.reshape(hidden_E_next, [hidden_D_next.size,1])


In [18]:
# model_F.fit(
#     x=hidden_F,
#     y=hidden_F_next,
#     epochs = 100,
#     batch_size=325,
#     use_multiprocessing=True
# )


manera de fer que tot estigui com hauria d'estar, s'ha de fer un reshape general del dataset segur 

donatt que el dataset original es segur [x,325] on x son els timesteps que tenim, es podrien tindre de manera senzilla amb una eina de tall o demanant el size [,1].size

fent la transposició ho fa tot molt més facil per operar, la matriu d'adjacencia s'hauria de modificar per si aca, crec


In [19]:
hidden_value_sans_names = hidden_value_all.iloc[:, 1:]

hidden_D_sans  = hidden_value_sans_names.to_numpy()
hidden_E_sans = tf.cast(hidden_D_sans,dtype=tf.float32)

shapes = hidden_E_sans.shape

noves_shapes = tf.reshape(hidden_E_sans, [shapes[1],shapes[0]])


In [20]:
def predictor_layer(name="Predictor"):
    predictor_layers =[]
    predictor_layers.append(keras.layers.Dense(64,activation='relu'))
    predictor_layers.append(keras.layers.Dense(32,activation='relu'))
    predictor_layers.append(keras.layers.Dense(1))

    return keras.Sequential(predictor_layers, name=name)

In [21]:
# def mlp_layer(name="mlp_placeholder"):
#     mlp_layers =[]
#     mlp_layers.append(keras.layers.Dense(,activation='sigmoid'))
#     mlp.layers.append(keras.layers.Dense(,activation='sigmoid'))
#     mlp.layers.append(keras.layers.Dense(,activation='sigmoid'))

#     return keras.Sequential(mlp_layers, name=name)

In [22]:
class STGNN(tf.keras.Model):
    def __init__(
        self,
        graph_info, #matriu d'adjecencia del graph
        hidden_units, #quantes features estarem tractant per timestep i sensor
        aggregation_type="sum",
        combination_type="gru",
        dropout_rate=0.2,
        normalize=False,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        # Unpack graph_info to three elements: node_features, edges, and edge_weight.
        edges, edge_weights = graph_info
        self.edges = edges
        self.edge_weights = edge_weights
        # Set edge_weights to ones if not provided.
        if self.edge_weights is None:
            self.edge_weights = tf.ones(shape=edges.shape[1])
        # Scale edge_weights to sum to 1.
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)
        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.normalize = normalize
        self.hidden_units = hidden_units


        # Create a process layer.
        self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
        # Create the first GraphConv layer.
        self.conv1 = GCN.GraphConvLayer(
            hidden_units,
            dropout_rate,
            aggregation_type,
            combination_type,
            normalize,
            name="graph_conv1",
        )
        # Create a postprocess layer.
        # self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        # self.compute_logits = layers.Dense(units=num_classes, name="logits")

        self.grucell = tf.keras.layers.GRUCell(64,name="Gru_temporal")

        # self.MLP = mlp_layer()

        #self.NovaGRU = tf.keras.layers.GRU(64,name="Gru_final")
        
        self.tbloc = tran.TransformerBlock(
            embed_dim=64, 
            num_heads=4, 
            ff_dim=64
        )

        
        self.predictor = predictor_layer()
        # la prediction layer es una multi feed-forward network per predir veloocitat transit

        
    
    

    def call(self, input_node_features_all):
        # Fem una petita modificació, ara entra tot el dataset de cop (52116, 325)
        # pero igualment els primers tractaments son de timestep a timestep
        # per tant primer s'itera per alla i finalment s'aglomera tot i es fa el predictor

        # Fare un apetita consideracio, nomes plantejare un batch de [1,12,325]
        aux = input_node_features_all[0]
        # print(old_x.shape)
        zerox=aux[0]
        # tf.print("primera valua per a mes tard old_x: ", old_x.shape)
        old_x = tf.expand_dims(zerox,axis=1)
        # tf.print("reshape de old_x fet", old_x.shape)
        old_x = self.preprocess(old_x)
        # tf.print("prepo old_x fet_:", old_x.shape)
        x_full= tf.expand_dims(old_x,axis=1)
        # tf.print("Hem expandit dimensioins i entrarem al loop")
        iter_count=0
        # tf.print(x_full.shape)
        #mirar si es correcte la manera en que accedim a les dades?
        # tf.print(input_node_features_all.shape)
        for input_node_features in aux[1:] :
            
            tf.autograph.experimental.set_loop_options(
            shape_invariants=[(x_full, tf.TensorShape([x_full.shape[0],None,x_full.shape[2]]))]
            )
            # Preprocess the node_features to produce node representation
            x= tf.expand_dims(input_node_features,axis=1)
            # x = tf.reshape(input_node_features,[input_node_features.shape[1],input_node_features.shape[0]])
            # tf.print(x.shape)
            x = self.preprocess(x)
            # Apply the first graph conv layer.
            x1 = self.conv1((x, self.edges, self.edge_weights))
            # tf.print("Despres de la convo: ", x1.shape)
            # Skip connection.
            x = x1 + x
            # Apply the second graph conv layer.
            # Postprocess node embedding.
            x, x2 = self.grucell(x,old_x) 
            # tf.print("post grucell: ", x2.shape)
            old_x = x2
            # tf.print("x_full shape:",x_full.shape)
            x_full = tf.concat([x_full,tf.expand_dims(old_x,axis=1)],1)
            # tf.print("x_full despres del concat x_full:", x_full.shape)
            iter_count +=1
        

        
        x_full = tf.convert_to_tensor(x_full,tf.float32)
        # tf.print("x_full despres d'un bucle per si un cas vull veure que passa",x_full.shape)
        # tf.print(x_full.shape)
        #res = self.NovaGRU(x_full)
        # tf.print(x_full.shape) #325, 12, 64
        # sortiran bastant més valors del transformer, es pot provar amb i o sense transformer

        res = self.tbloc(x_full,x_full) #hauria de treure 325, 64
        # tf.print("sortida del transformer: ", res.shape)
        # tf.print(res.shape)

        x = self.predictor(res) # hauria de treure 325, 12 SEWMPRE una sortida
        
        x=tf.squeeze(x)
        
        x= tf.transpose(x)
        return x
 


In [23]:
model_Full = STGNN(    
    graph_info=(edges, a_p.data),
    hidden_units=[64],
    dropout_rate=0.2,
    name="Spatio_temporal_model")

In [24]:
hidden_M_sans  = hidden_value_sans_names.iloc[:5,:]

alll = tf.cast(hidden_value_sans_names,dtype=tf.float32)

mitja_Set = tf.cast(hidden_M_sans,dtype=tf.float32)

next_M_sans  = hidden_value_sans_names.iloc[5:10,:]



nexting_set = tf.cast(next_M_sans,dtype=tf.float32)

mitja_SET = tf.reshape(mitja_Set,[mitja_Set.shape[1],mitja_Set.shape[0]])
nexting_SET = tf.reshape(nexting_set,[nexting_set.shape[1],nexting_set.shape[0]])


 #enviar en troços petits

In [25]:
alll_t  = tf.transpose(alll)

win_size=12

x_train = []
y_train = []

for i in range(win_size, alll.shape[0]-win_size):
    x_train.append(alll[i-win_size:i, :])
    y_train.append(alll[i:win_size+i, :])

x_train, y_train = np.array(x_train), np.array(y_train)
train_y=np.reshape(y_train,(y_train.shape[0],y_train.shape[2],y_train.shape[1]))
print(train_y.shape)

(52092, 325, 12)


In [26]:
print(model_Full(x_train[:1]))

tf.Tensor(
[[ 0.35053957  0.3488285   0.35009933 ...  0.34912318  0.35017252
   0.34885836]
 [-0.14089811 -0.13997126 -0.14109683 ... -0.13606536 -0.13291311
  -0.1341443 ]
 [-0.8953564  -0.8902283  -0.8926969  ... -0.89045537 -0.8917784
  -0.8905465 ]
 ...
 [-1.419546   -1.3566755  -1.4073583  ... -1.3753352  -1.414859
  -1.3601422 ]
 [-1.3992493  -1.3356683  -1.3803463  ... -1.3520825  -1.3913398
  -1.3382882 ]
 [-1.3744205  -1.3088739  -1.3585227  ... -1.3279381  -1.3680899
  -1.3122844 ]], shape=(12, 325), dtype=float32)


In [27]:

# model_Full.build(input_shape=[5,325])
print(model_Full.summary())

Model: "Spatio_temporal_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocess (Sequential)      (325, 64)                 4292      
_________________________________________________________________
graph_conv1 (GraphConvLayer) multiple                  8832      
_________________________________________________________________
Gru_temporal (GRUCell)       multiple                  24960     
_________________________________________________________________
transformer_block (Transform multiple                  74944     
_________________________________________________________________
Predictor (Sequential)       (325, 12, 1)              6273      
Total params: 119,301
Trainable params: 119,043
Non-trainable params: 258
_________________________________________________________________
None


In [28]:

alll_t  = tf.transpose(alll)

win_size=12

x_train = []
y_train = []

for i in range(win_size, alll.shape[0]-win_size):
    x_train.append(alll[i-win_size:i, :])
    y_train.append(alll[i:win_size+i, :])

x_train, y_train = np.array(x_train), np.array(y_train)
train_y=np.reshape(y_train,(y_train.shape[0],y_train.shape[2],y_train.shape[1]))
print(train_y.shape)

(52092, 325, 12)


In [29]:
x_test=x_train[48000:]
y_test=y_train[48000:]
print(y_test.shape)

(4092, 12, 325)


In [30]:
# Prepare the test data
x_test = []
y_test = []

for i in range(win_size, alll.shape[0]-win_size):
    x_test.append(alll[i-win_size:i, :])
    y_test.append(alll[i:win_size+i, :])

x_test, y_test = np.array(x_test), np.array(y_test)
print(y_test.shape)

(52092, 12, 325)


In [31]:
%load_ext tensorboard

In [32]:

x_test=x_test[48000:]
y_test=y_test[48000:]

test_x=np.reshape(x_test,(x_test.shape[0],x_test.shape[2],x_test.shape[1]))
test_y=np.reshape(y_test,(y_test.shape[0],y_test.shape[2],y_test.shape[1]))
print(test_y.shape)

(4092, 325, 12)


In [33]:
# data = data.repeat(5)


# train=data.take(200000)


es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.001, patience=5, restore_best_weights=True)

tensorboard =  tf.keras.callbacks.TensorBoard(log_dir="logs")



model_Full.compile(optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.MeanAbsoluteError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae"),tf.keras.metrics.MeanAbsolutePercentageError(name="mape"),tf.keras.metrics.RootMeanSquaredError(name="rmse")])
# modelckpt_callback = keras.callbacks.ModelCheckpoint(monitor="val_loss",filepath="sub_wheights_"+str(rate)+".hdf5",verbose=1,save_weights_only=True,save_best_only=True)
filepath = os.path.join("pesos_12a12_v2", "{epoch:02d}-{val_loss:.4f}")
modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=filepath,
verbose=1,
mode="min",
monitor='val_loss',
save_best_only=False,
save_weights_only=True,
save_freq='epoch')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.5,
                                                patience=10, verbose=1)



model_Full.fit(
    x=x_train,
    y=y_train,
    batch_size=1,
    epochs = 20, 
    steps_per_epoch=5000,
    validation_data=(x_test,y_test),
    validation_steps=4000,
    use_multiprocessing=True,
    callbacks =[modelckpt_callback,tensorboard,reduce_lr]
    )

# Recrear la taula d'experiments, direcament fer-ho a lo complert
# !!!!!!!!!!!!!!!!!!!!Fes la puta acumulació de la taula!!!!!!!!!!!!!!!!!
# Alguna cosa no va prou bé aqui, no es un problema que la sortida sigui un 352,12 i els de de validacio sirin 12, 352?
# He intentat flipejar la sortida fent un reshape i mesta petant bastant

Epoch 1/20
5000/5000 [==============================] - 500s 99ms/step - loss: 3.9443 - mae: 3.9443 - mape: 752515.8125 - rmse: 8.0352 - val_loss: 4.9219 - val_mae: 4.9219 - val_mape: 12.2398 - val_rmse: 8.0758

Epoch 00001: saving model to pesos_12a12_v2\01-4.9219
Epoch 2/20
5000/5000 [==============================] - 493s 99ms/step - loss: 2.7294 - mae: 2.7294 - mape: 1327363.0000 - rmse: 5.2119 - val_loss: 4.5008 - val_mae: 4.5008 - val_mape: 10.6943 - val_rmse: 7.0598

Epoch 00002: saving model to pesos_12a12_v2\02-4.5008
Epoch 3/20
5000/5000 [==============================] - 487s 97ms/step - loss: 2.6220 - mae: 2.6220 - mape: 1285579.0000 - rmse: 5.0366 - val_loss: 5.1369 - val_mae: 5.1369 - val_mape: 13.1694 - val_rmse: 8.7508

Epoch 00003: saving model to pesos_12a12_v2\03-5.1369
Epoch 4/20
5000/5000 [==============================] - 489s 98ms/step - loss: 2.4989 - mae: 2.4989 - mape: 705747.0625 - rmse: 4.8911 - val_loss: 4.6585 - val_mae: 4.6585 - val_mape: 11.4593 - val_rm

In [ ]:
model_Full.save_weights("./pesos_bastant_entrenats")

In [ ]:
model_Full.load_weights("pesos_TRanformed_modificats_a12/18-1.1050")

In [87]:
cpmar = np.concatenate([model_Full.predict(x_train[:1]),
# tf.reshape(alll[12:24],[alll[12:24].shape[1],alll[12:24].shape[0]])]
alll[12:24]],
axis=1)
print(cpmar[0])


# plot.plot(cpmar,label=["pred","real"]) #Per un moment concret es el que tenim
# plot.legend()
# plot.xlabel("Nº sensors")
# plot.ylabel("Q metric")
# plot.show()
# plot de coses com aquestes, queda maco

[65.0531   65.02501  65.338776 64.93062  64.983284 65.18354  65.319336
 65.23681  64.95065  64.887695 65.268585 65.190926 64.60709  64.53182
 64.581345 64.9641   64.593155 64.881905 65.082405 64.96498  65.02519
 65.1099   65.34745  65.511314 64.37474  64.27281  64.146935 65.04835
 64.39915  64.7403   64.98682  64.850685 65.13344  65.30791  65.450874
 65.76963  64.725914 64.6608   64.74747  65.0159   64.70739  64.975624
 65.17336  65.06222  65.0718   65.123055 65.38756  65.51209  64.45816
 64.33806  64.14024  65.25042  64.49821  64.84479  65.101555 64.96253
 65.348045 65.54365  65.65149  66.026474 64.55637  64.48367  64.54231
 64.90622  64.547966 64.834    65.03849  64.91592  64.973816 65.053055
 65.3026   65.46352  64.82183  64.741615 64.757744 65.22194  64.81103
 65.090614 65.28702  65.18443  65.27668  65.360504 65.573395 65.75393
 64.46131  64.37153  64.33675  64.97182  64.46739  64.78233  65.006226
 64.87582  65.04979  65.17653  65.37312  65.61319  64.3685   64.2129
 63.76269  65.58

In [ ]:
valors_sueltos = []

for x in range(100):

    cpmar = np.concatenate([model_Full.predict(alll[0+x:12+x]),np.expand_dims(alll[x+12],axis=1)],axis=1)
    valors_sueltos.append(cpmar[135])

plot.title("100 timesteps, sensor 135")
plot.plot(valors_sueltos,label=["pred","real"]) #Per un moment concret es el que tenim
plot.legend()
plot.xlabel("timestep")
plot.ylabel("Q metric")
plot.show()

In [ ]:
pas_mig = model_F(hidden_F)
pas_segon = model_F(hidden_F_next)
#els nputs d'una gru son els seguents [batch, timesteps, feature]
#la seguent activacio es una simple prova de funcionament
inputs = tf.random.normal([22, 12, 6])

le_gru = tf.keras.layers.GRU(1)
# per cada gru pasar el valor de cada node per separat, despress reajuntar
# en la primera podem considerar estat 0 pre gcn, features pures del node, i el estat 1 post gcn 
le_cell = tf.keras.layers.GRUCell(1)

s_1, s_2 = le_cell(pas_mig, hidden_F)

b_1, b_2 = le_cell(pas_segon, s_2)
